<a href="https://colab.research.google.com/github/Scott-Huston/DS-Unit-2-Regression-Classification/blob/master/assignment_regression_classification_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Regression & Classification, Module 3

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

Instead, predict property sales prices for **One Family Dwellings** (`BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'`) using a subset of the data where the **sale price was more than \\$100 thousand and less than $2 million.** 

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


- [X] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [X] Do exploratory visualizations with Seaborn.
- [X] Do one-hot encoding of categorical features.
- [X] Do feature selection with `SelectKBest`.
- [X] Fit a linear regression model with multiple features.
- [X] Get mean absolute error for the test set.
- [X] As always, commit your notebook to your fork of the GitHub repo.


## Stretch Goals
- [ ] Add your own stretch goal(s) !
- [X] Do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way (without an excessive amount of formulas or academic pre-requisites).
(That book is good regardless of whether your cultural worldview is inferential statistics or predictive machine learning)
- [ ] Read Leo Breiman's paper, ["Statistical Modeling: The Two Cultures"](https://projecteuclid.org/download/pdf_1/euclid.ss/1009213726)
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html):

> Pipeline can be used to chain multiple estimators into one. This is useful as there is often a fixed sequence of steps in processing the data, for example feature selection, normalization and classification. Pipeline serves multiple purposes here:

> - **Convenience and encapsulation.** You only have to call fit and predict once on your data to fit a whole sequence of estimators.
> - **Joint parameter selection.** You can grid search over parameters of all estimators in the pipeline at once.
> - **Safety.** Pipelines help avoid leaking statistics from your test data into the trained model in cross-validation, by ensuring that the same samples are used to train the transformers and predictors.

In [1]:
# If you're in Colab...
import os, sys
in_colab = 'google.colab' in sys.modules

if in_colab:
    # Install required python packages:
    # category_encoders, version >= 2.0
    # pandas-profiling, version >= 2.0
    # plotly, version >= 4.0
    !pip install --upgrade category_encoders pandas-profiling plotly
    
    # Pull files from Github repo
    os.chdir('/content')
    !git init .
    !git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification.git
    !git pull origin master
    
    # Change into directory for module
    os.chdir('module3')

Requirement already up-to-date: category_encoders in /usr/local/lib/python3.6/dist-packages (2.0.0)
Requirement already up-to-date: pandas-profiling in /usr/local/lib/python3.6/dist-packages (2.3.0)
Requirement already up-to-date: plotly in /usr/local/lib/python3.6/dist-packages (4.1.0)
Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
From https://github.com/LambdaSchool/DS-Unit-2-Regression-Classification
 * branch            master     -> FETCH_HEAD
Already up to date.


In [0]:
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv('../data/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [4]:
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
0,1,CHELSEA,13 CONDOS - ELEVATOR APARTMENTS,2,716,1246,NaN,R4,"447 WEST 18TH STREET, PH12A",PH12A,10011.0,1.0,0.0,1.0,"10,733",1979.0,2007.0,2,R4,0,01/01/2019
1,1,FASHION,21 OFFICE BUILDINGS,4,812,68,NaN,O5,144 WEST 37TH STREET,NaN,10018.0,0.0,6.0,6.0,"2,962",15435.0,1920.0,4,O5,0,01/01/2019
2,1,FASHION,21 OFFICE BUILDINGS,4,839,69,NaN,O5,40 WEST 38TH STREET,NaN,10018.0,0.0,7.0,7.0,"2,074",11332.0,1930.0,4,O5,0,01/01/2019
3,1,GREENWICH VILLAGE-WEST,13 CONDOS - ELEVATOR APARTMENTS,2,592,1041,NaN,R4,"1 SHERIDAN SQUARE, 8C",8C,10014.0,1.0,0.0,1.0,0,500.0,0.0,2,R4,0,01/01/2019
4,1,UPPER EAST SIDE (59-79),15 CONDOS - 2-10 UNIT RESIDENTIAL,2C,1379,1402,NaN,R1,"20 EAST 65TH STREET, B",B,10065.0,1.0,0.0,1.0,0,6406.0,0.0,2,R1,0,01/01/2019


In [5]:
# Filtering for only one family dwellings
df = df[df['BUILDING_CLASS_CATEGORY']=='01 ONE FAMILY DWELLINGS']
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
7,2,MORRIS PARK/VAN NEST,01 ONE FAMILY DWELLINGS,1,4090,37,NaN,A1,1193 SACKET AVENUE,NaN,10461.0,1.0,0.0,1.0,"3,404",1328.0,1925.0,1,A1,0,01/01/2019
8,2,MORRIS PARK/VAN NEST,01 ONE FAMILY DWELLINGS,1,4120,18,NaN,A5,1215 VAN NEST AVENUE,NaN,10461.0,1.0,0.0,1.0,"2,042",1728.0,1935.0,1,A5,0,01/01/2019
9,2,MORRIS PARK/VAN NEST,01 ONE FAMILY DWELLINGS,1,4120,20,NaN,A5,1211 VAN NEST AVENUE,NaN,10461.0,1.0,0.0,1.0,"2,042",1728.0,1935.0,1,A5,0,01/01/2019
42,3,MADISON,01 ONE FAMILY DWELLINGS,1,6809,54,NaN,A1,2601 AVENUE R,NaN,11229.0,1.0,0.0,1.0,"3,333",1262.0,1925.0,1,A1,0,01/01/2019
44,3,OCEAN PARKWAY-NORTH,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019


In [6]:
# filtering where sale price was more than $100 thousand and less than $2 million.

df = df[(df['SALE_PRICE']<2000000) & (df['SALE_PRICE']>100000)]
df.head()

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OCEAN PARKWAY-NORTH,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,QUEENS VILLAGE,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
78,2,PELHAM PARKWAY SOUTH,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,01/02/2019
108,3,FLATBUSH-CENTRAL,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,01/02/2019
111,3,FLATBUSH-EAST,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,01/02/2019


In [7]:
# Splitting training and test data

df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'], infer_datetime_format = True)
train = df[(df['SALE_DATE'].dt.month==3) & (df['SALE_DATE'].dt.year ==2019)]
test = df[(df['SALE_DATE'].dt.month==4) & (df['SALE_DATE'].dt.year ==2019)]

# Checking that it worked correctly
print(train.SALE_DATE.describe())
print(test.SALE_DATE.describe())

count                     799
unique                     24
top       2019-03-29 00:00:00
freq                       62
first     2019-03-01 00:00:00
last      2019-03-30 00:00:00
Name: SALE_DATE, dtype: object
count                     644
unique                     23
top       2019-04-15 00:00:00
freq                       43
first     2019-04-01 00:00:00
last      2019-04-30 00:00:00
Name: SALE_DATE, dtype: object


In [8]:
train.LAND_SQUARE_FEET.isnull().sum()

0

In [9]:
# Included as part of feature engineering function
# train.LAND_SQUARE_FEET = train.LAND_SQUARE_FEET.str.replace(',','')
# train['LAND_SQUARE_FEET'] = train['LAND_SQUARE_FEET'].astype(int)
train.LAND_SQUARE_FEET.dtypes


dtype('O')

In [10]:
train.BUILDING_CLASS_AT_PRESENT.describe()

count     799
unique     10
top        A1
freq      284
Name: BUILDING_CLASS_AT_PRESENT, dtype: object

In [11]:
train.NEIGHBORHOOD.describe()

count                799
unique               153
top       FLUSHING-NORTH
freq                  26
Name: NEIGHBORHOOD, dtype: object

In [12]:
# All of the values are null in the EASE-MENT, and APARTMENT_NUMBER features
# (probably due to privacy concerns)
print(train.isnull().sum())

# Dropping null columns
# included as part of feature engineering function
# train.drop(['EASE-MENT', 'APARTMENT_NUMBER'], axis = 'columns', inplace = True)

train.isnull().sum()


BOROUGH                             0
NEIGHBORHOOD                        0
BUILDING_CLASS_CATEGORY             0
TAX_CLASS_AT_PRESENT                0
BLOCK                               0
LOT                                 0
EASE-MENT                         799
BUILDING_CLASS_AT_PRESENT           0
ADDRESS                             0
APARTMENT_NUMBER                  799
ZIP_CODE                            0
RESIDENTIAL_UNITS                   0
COMMERCIAL_UNITS                    0
TOTAL_UNITS                         0
LAND_SQUARE_FEET                    0
GROSS_SQUARE_FEET                   0
YEAR_BUILT                          0
TAX_CLASS_AT_TIME_OF_SALE           0
BUILDING_CLASS_AT_TIME_OF_SALE      0
SALE_PRICE                          0
SALE_DATE                           0
dtype: int64


BOROUGH                             0
NEIGHBORHOOD                        0
BUILDING_CLASS_CATEGORY             0
TAX_CLASS_AT_PRESENT                0
BLOCK                               0
LOT                                 0
EASE-MENT                         799
BUILDING_CLASS_AT_PRESENT           0
ADDRESS                             0
APARTMENT_NUMBER                  799
ZIP_CODE                            0
RESIDENTIAL_UNITS                   0
COMMERCIAL_UNITS                    0
TOTAL_UNITS                         0
LAND_SQUARE_FEET                    0
GROSS_SQUARE_FEET                   0
YEAR_BUILT                          0
TAX_CLASS_AT_TIME_OF_SALE           0
BUILDING_CLASS_AT_TIME_OF_SALE      0
SALE_PRICE                          0
SALE_DATE                           0
dtype: int64

In [0]:
# Exploratory visualizations

# import seaborn as sns

# sns.pairplot(train, plot_kws= dict(alpha=.05));

In [0]:
# Filtering training data further to only look at residential units
train = train[train.RESIDENTIAL_UNITS==1]


In [15]:
# Thinking about doing one-hot-encoding on zip code, but that's quite a few unique zip codes
train.ZIP_CODE.nunique()

109

In [0]:
# Grouping zip codes by ignoring last digit
# train['ROUNDED_ZIP'] = train['ZIP_CODE']//10
# Now there are 20 unique groups, that's better
# train.ROUNDED_ZIP.nunique()

In [17]:
train.describe(exclude = 'number')

,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,LAND_SQUARE_FEET,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
count,787,787,787,787,787,0.0,787,787,787
unique,152,1,1,9,786,0.0,363,9,24
top,FLUSHING-NORTH,01 ONE FAMILY DWELLINGS,1,A1,117-45 125TH STREET,NaN,"4,000",A1,2019-03-29 00:00:00
freq,26,787,787,284,2,NaN,89,284,62
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-01 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-30 00:00:00


In [0]:
# Creating function for feature engineering
def feature_engineering(df):
  import category_encoders as ce
  
#   dropping null columns
  df.drop(['EASE-MENT', 'APARTMENT_NUMBER'], axis = 'columns', inplace = True)
  
  #  Get rounded zip feature
  df['ROUNDED_ZIP'] = df['ZIP_CODE']//10
  
  # Change land square feet to an int
  df.LAND_SQUARE_FEET = df.LAND_SQUARE_FEET.str.replace(',','')
  df['LAND_SQUARE_FEET'] = df['LAND_SQUARE_FEET'].astype(int)
  
  # one-hot-encoding building_class_at_present and rounded_zip
  encoder = ce.OneHotEncoder(cols = ['BUILDING_CLASS_AT_TIME_OF_SALE', 'ROUNDED_ZIP'], use_cat_names = True)
  encoded = encoder.fit_transform(df)
  
  return encoded
  
  

  
 

  

In [0]:
from sklearn.feature_selection import f_regression, SelectKBest
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

target = 'SALE_PRICE'
exclude = ['NEIGHBORHOOD', 'BUILDING_CLASS_CATEGORY', 'TAX_CLASS_AT_PRESENT', 'BUILDING_CLASS_AT_PRESENT','ADDRESS','SALE_DATE']


In [0]:
# train = feature_engineering(train)

import category_encoders as ce
  
#   dropping null columns
train.drop(['EASE-MENT', 'APARTMENT_NUMBER'], axis = 'columns', inplace = True)
  
  #  Get rounded zip feature
train['ROUNDED_ZIP'] = train['ZIP_CODE']//10
  
  # Change land square feet to an int
train.LAND_SQUARE_FEET = train.LAND_SQUARE_FEET.str.replace(',','')
train['LAND_SQUARE_FEET'] = train['LAND_SQUARE_FEET'].astype(int)
  
  # one-hot-encoding building_class_at_present and rounded_zip
encoder = ce.OneHotEncoder(cols = ['BUILDING_CLASS_AT_TIME_OF_SALE', 'ROUNDED_ZIP'], use_cat_names = True)
train = encoder.fit_transform(train)
  



In [0]:
# test = feature_engineering(test)

#   dropping null columns
test = test.drop(['EASE-MENT', 'APARTMENT_NUMBER'], axis = 'columns')

In [0]:
  #  Get rounded zip feature
test['ROUNDED_ZIP'] = test['ZIP_CODE']//10

In [0]:
  # Change land square feet to an int
test.LAND_SQUARE_FEET = test.LAND_SQUARE_FEET.str.replace(',','')
test['LAND_SQUARE_FEET'] = test['LAND_SQUARE_FEET'].astype(int)

In [0]:
  # one-hot-encoding building_class_at_present and rounded_zip
encoder = ce.OneHotEncoder(cols = ['BUILDING_CLASS_AT_TIME_OF_SALE', 'ROUNDED_ZIP'], use_cat_names = True)
test = encoder.fit_transform(test)

In [0]:
features = train.columns.drop([target] + exclude)

In [0]:
X_train = train[features]
y_train = train[target]

In [0]:
X_test = test[features]
y_test = test[target]

In [28]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train.head()

,BOROUGH,BLOCK,LOT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE_A5,BUILDING_CLASS_AT_TIME_OF_SALE_A1,BUILDING_CLASS_AT_TIME_OF_SALE_A9,BUILDING_CLASS_AT_TIME_OF_SALE_A4,BUILDING_CLASS_AT_TIME_OF_SALE_A2,BUILDING_CLASS_AT_TIME_OF_SALE_A3,BUILDING_CLASS_AT_TIME_OF_SALE_S1,BUILDING_CLASS_AT_TIME_OF_SALE_A0,BUILDING_CLASS_AT_TIME_OF_SALE_A6,ROUNDED_ZIP_1046.0,ROUNDED_ZIP_1047.0,ROUNDED_ZIP_1123.0,ROUNDED_ZIP_1120.0,ROUNDED_ZIP_1122.0,ROUNDED_ZIP_1169.0,ROUNDED_ZIP_1141.0,ROUNDED_ZIP_1136.0,ROUNDED_ZIP_1135.0,ROUNDED_ZIP_1137.0,ROUNDED_ZIP_1142.0,ROUNDED_ZIP_1143.0,ROUNDED_ZIP_1031.0,ROUNDED_ZIP_1030.0,ROUNDED_ZIP_1121.0,ROUNDED_ZIP_1100.0,ROUNDED_ZIP_1110.0,ROUNDED_ZIP_1045.0,ROUNDED_ZIP_1104.0,ROUNDED_ZIP_1138.0
12115,2,4903,75,10466.0,1.0,0.0,1.0,1840,1584.0,1970.0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12116,2,3791,44,10472.0,1.0,0.0,1.0,1854,1714.0,1955.0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12124,2,4483,30,10469.0,1.0,0.0,1.0,2600,2110.0,1950.0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12127,2,5334,45,10461.0,1.0,0.0,1.0,4915,2400.0,1930.0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
12130,2,5436,38,10465.0,1.0,0.0,1.0,5500,1800.0,1920.0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [30]:
for k in range(1, len(X_train.columns)+1):
  print(f'{k} features')
  
  selector = SelectKBest(score_func = f_regression, k = k)
  selector.fit(X_train_scaled, y_train)
  X_train_selected = selector.transform(X_train_scaled)
 
  model = LinearRegression()
  model.fit(X_train_selected, y_train)
  
  y_pred = model.predict(X_train_selected)
  mae = mean_absolute_error(y_train, y_pred)
  print(f'Mean Absolute Error: {mae:,.2f}')
  

1 features
Mean Absolute Error: 187,427.98
2 features
Mean Absolute Error: 187,005.21
3 features
Mean Absolute Error: 185,052.03
4 features
Mean Absolute Error: 175,205.34
5 features
Mean Absolute Error: 168,846.57
6 features
Mean Absolute Error: 165,906.76
7 features
Mean Absolute Error: 162,822.59
8 features
Mean Absolute Error: 156,273.39
9 features
Mean Absolute Error: 150,341.62
10 features
Mean Absolute Error: 149,771.42
11 features
Mean Absolute Error: 149,531.48
12 features
Mean Absolute Error: 149,829.24
13 features
Mean Absolute Error: 149,257.09
14 features
Mean Absolute Error: 149,179.69
15 features
Mean Absolute Error: 149,142.90
16 features
Mean Absolute Error: 149,024.30
17 features
Mean Absolute Error: 148,680.05
18 features
Mean Absolute Error: 148,282.57
19 features
Mean Absolute Error: 148,239.38
20 features
Mean Absolute Error: 147,744.64
21 features
Mean Absolute Error: 145,869.04
22 features
Mean Absolute Error: 145,945.03
23 features
Mean Absolute Error: 145,970.

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/p

Mean Absolute Error: 140,986.66


In [32]:
# Fitting model with 34 best features

selector = SelectKBest(score_func = f_regression, k = 34)
X_train_selected = selector.fit_transform(X_train_scaled, y_train)
X_test_selected = selector.transform(X_test_scaled)

model = LinearRegression()
model.fit(X_train_selected, y_train)

# Getting test mean average error
y_pred = model.predict(X_test_selected)
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae:,.2f}')

Mean Absolute Error: 161,812.12


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/univariate_selection.py:299: RuntimeWarning: invalid value encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:901: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1892: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
